In [19]:
import scipy.io
import numpy as np
import pandas as pd
def load_mat(var):
    file_name ="/home/roseyu/Data/combustion/"+var+".mat"
    array =scipy.io.loadmat(file_name)
    a_val = array[var]
    return a_val

### turn a 3-d ndarray to a movie clip ( time x width x height )

In [ ]:
import moviepy.editor as mpy
import matplotlib.pyplot as plt
def rescale_arr_to_255(m):
    _min = np.amin(m)
    _max = np.amax(m)
    rgb_scale = 255.
    _m = m
    _m = m + _min
    _m = _m / (_max - _min)
    _m = _m * rgb_scale
    return _m.astype(int)

def scale(f, factor=1):
    from numpy.lib.stride_tricks import as_strided
    M, N = factor,factor # number of time to replicate each point in each dimension
    C, H, W = f.shape
    _f = as_strided(f,
    (C, 1, H, M, W, N),
    (f.strides[0], 0, f.strides[1], 0, f.strides[2], 0))
    _f = _f.reshape((C, H*M, W*N))
    return _f

def create_clip(frames, fps, factor):

    T = frames.shape[0]
    duration = T / fps
    print("Creating clip of {} timesteps = {} secs".format(T, duration))

    def make_frame(t):
        _t = min(int(t * fps), T - 1)
        f = frames[_t]
        f = rescale_arr_to_255(f)
        f = scale(f, factor=factor)
        f = np.transpose(f, (1, 2, 0))
        return f
#     fig, ax = plt.subplots()
#     def make_frame(t):
#         ax.clear()
#         _t = min(int(t * fps), T - 1)
#         ax.imshow(a_val[_t,:,:])
# #         ax.set_ylim(-1.5, 2.5)
#         return mplfig_to_npimage(fig)


    clip = mpy.VideoClip(make_frame=make_frame, duration=duration)
    clip.duration = duration
    clip.fps = fps
    clip.size = factor*frames[0].shape[2],factor*frames[0].shape[1]
    return clip

In [ ]:
def array2movie(var):
    a = load_mat(var)
    frames = np.expand_dims(a, 1)
    fps, factor = 60, 1
    movie = create_clip(frames, fps, factor)
    return movie

In [ ]:
%matplotlib inline
clip_U = array2movie('U')
clip_U.ipython_display(fps=20, loop=0, autoplay=1)

In [ ]:
clip_V = array2movie('V')
clip_V.ipython_display(fps=20, loop=0, autoplay=1)

In [ ]:
clip_V = array2movie('P')
clip_V.ipython_display(fps=20, loop=0, autoplay=1)

In [ ]:
clip_V = array2movie('W')
clip_V.ipython_display(fps=20, loop=0, autoplay=1)

### evenly sample time series from the 3-d array

In [22]:
def sample_ts(var, nx, ny, verbose=False):
    a = load_mat(var)
    lx = a.shape[1]
    ly = a.shape[1]
    print('x dim', lx, 'y dim', ly)
    x = np.linspace(0,lx-1, nx, dtype=int)
    y = np.linspace(0,ly-1, ny, dtype=int)
    xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')
    a_r = np.empty((a.shape[0],0))
    # treat xv[i,j], yv[i,j]
    for i in range(nx):
        for j in range(ny):
            if verbose:
                print('pose:',xv[i,j], yv[i,j])

            a_r = np.append(a_r, np.expand_dims(a[:,xv[i,j], yv[i,j]],axis=1), axis =1)
    return a_r

def rescale_ts(ts):
    df = pd.DataFrame(ts)
    df_norm = (df - df.mean()) / (df.max() - df.min())
    ts_norm = df_norm.as_matrix()
    return ts_norm

### TODO: the time series length for combusion is too short

In [29]:
ts_list= []
for var in ['U','V','W','P']:
    ts = sample_ts(var, 4, 4)
    print(var, ' shape:',ts.shape)
    ts = rescale_ts(ts)
    ts_list += [ts]

ts_all = np.concatenate(ts_list, axis=1)
print(ts_all.shape)
    
# save_path = '/home/roseyu/Data/TensorRNN/'+ var
# np.save(save_path, ts)
# print('saved to '+ save_path)
# import pickle
# with open('/home/roseyu/data/tensorRNN/'+ var+ '.pkl', 'wb') as f:
#     pickle.dump(ts, f)

('x dim', 128, 'y dim', 128)
('U', ' shape:', (1408, 16))
('x dim', 128, 'y dim', 128)
('V', ' shape:', (1408, 16))
('x dim', 128, 'y dim', 128)
('W', ' shape:', (1408, 16))
('x dim', 128, 'y dim', 128)
('P', ' shape:', (1408, 16))
(1408, 64)


In [ ]:
plt.plot(ts[:,20])

### rescale the time series to be [-1,1]

In [ ]:


file_name = '/home/roseyu/Data/tensorRNN/U.pkl'

with open(file_name,'wb') as f:
     pickle.dump(ts_norm,f)
plt.plot(ts_norm[:,20])